In [3]:
import pandas as pd
import geopandas as gpd
import json

In [1]:
def generate_unique_id(df, county_col, prec_code_col, unique_id_col='UNIQUE_ID'):
    """
    Generates a unique identifier for each row in the DataFrame based on county and precinct code.
    Appends a suffix to resolve duplicates.

    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        county_col (str): The column name for the county.
        prec_code_col (str): The column name for the precinct code.
        unique_id_col (str): The name of the new or existing unique identifier column.

    Returns:
        pd.DataFrame: The updated DataFrame with the unique ID column.
    """
    # Generate the initial UNIQUE_ID
    df[unique_id_col] = (
        df[county_col] + '_PRECINCT_' +
        df[prec_code_col].astype(str)
    )

    # Append suffix to resolve duplicates
    df[unique_id_col] = (
        df[unique_id_col] +
        '_' +
        df.groupby(unique_id_col).cumcount().astype(str).replace('0', '')
    )
    
    return df


## Preprocess SC Election result from Precincts

### Function and declaration

In [15]:
def process_precinct_votes(precinct_gdf, columns_to_use, columns_to_ignore):
    """
    Args:
        precinct_gdf (pd.DataFrame): The DataFrame containing precinct vote data.
        columns_to_use (list): The list of columns to consider for vote calculations.

    Returns:
        pd.DataFrame: The updated DataFrame with calculated vote totals, lean, and filtered party-related columns.
    """
    precinct_gdf['TOT_REP'] = 0
    precinct_gdf['TOT_DEM'] = 0
    precinct_gdf['TOT_VOT'] = 0

    republican_columns = []
    democratic_columns = []

    for col in columns_to_use:
        if len(col) >= 7:
            party_code = col[6]
            if party_code == 'R':  # Republican
                precinct_gdf['TOT_REP'] += precinct_gdf[col]
                republican_columns.append(col)
            elif party_code == 'D':  # Democrat
                precinct_gdf['TOT_DEM'] += precinct_gdf[col]
                democratic_columns.append(col)  

    precinct_gdf['TOT_VOT'] = (
        precinct_gdf['TOT_REP'] +
        precinct_gdf['TOT_DEM']
    )

    precinct_gdf['LEAN'] = precinct_gdf.apply(
        lambda row: 'Unknown' if row['TOT_REP'] == 0 and row['TOT_DEM'] == 0 else
                    ('Republican' if row['TOT_REP'] > row['TOT_DEM'] else
                     ('Democratic' if row['TOT_DEM'] > row['TOT_REP'] else 'Unknown')),
        axis=1
    )
    filtered_columns = columns_to_ignore + ['TOT_REP', 'TOT_DEM', 'TOT_VOT', 'LEAN'] + republican_columns + democratic_columns
    precinct_gdf = precinct_gdf[filtered_columns]

    return precinct_gdf


### Main script

#### Preprocess conegssional district election

In [4]:
sc_precincts_gdf = gpd.read_file('states/south_carolina/geodata/south_carolina_precincts.geojson')
sc_election_gdf = gpd.read_file('raw/precincts/sc_2022_gen_prec/sc_2022_gen_cong_prec/sc_2022_gen_cong_prec.shp')

In [5]:
sc_election_gdf = generate_unique_id(sc_election_gdf, county_col='County', prec_code_col='Prec_Code')


In [695]:
print(sc_precincts_gdf.columns)

Index(['UNIQUE_ID', 'NAME', 'Prec_Code', 'CONG_DIST', 'geometry'], dtype='object')


In [6]:
print(sc_precincts_gdf.head())

                 UNIQUE_ID             NAME Prec_Code CONG_DIST  \
0  ABBEVILLE_PRECINCT_001_  ABBEVILLE NO. 1       001        03   
1  ABBEVILLE_PRECINCT_002_  ABBEVILLE NO. 2       002        03   
2  ABBEVILLE_PRECINCT_003_  ABBEVILLE NO. 3       003        03   
3  ABBEVILLE_PRECINCT_004_  ABBEVILLE NO. 4       004        03   
4  ABBEVILLE_PRECINCT_005_       ANTREVILLE       005        03   

                                            geometry  
0  POLYGON ((-82.32898 34.18851, -82.32918 34.188...  
1  POLYGON ((-82.28839 34.09074, -82.28985 34.089...  
2  POLYGON ((-82.34099 34.10805, -82.34092 34.107...  
3  POLYGON ((-82.38745 34.18274, -82.38736 34.182...  
4  POLYGON ((-82.59624 34.30205, -82.58812 34.307...  


In [7]:
print(sc_election_gdf.head())

                  UNIQUE_ID COUNTYFP      County      Precinct Prec_Code  \
0  DORCHESTER_PRECINCT_084_      035  DORCHESTER  BEECH HILL 2       084   
1  DORCHESTER_PRECINCT_072_      035  DORCHESTER       CYPRESS       072   
2  DORCHESTER_PRECINCT_036_      035  DORCHESTER      DELEMARS       036   
3  DORCHESTER_PRECINCT_035_      035  DORCHESTER       GIVHANS       035   
4  DORCHESTER_PRECINCT_094_      035  DORCHESTER     GIVHANS 2       094   

  CONG_DIST  GCON01AODD  GCON01DAND  GCON01OWRI  GCON01RMAC  ...  GCON05GGAI  \
0        01          10         286           3         540  ...           0   
1        01          16         479           2         682  ...           0   
2        01           2          69           2          85  ...           0   
3        01           4         144           1         296  ...           0   
4        01           8         189           0         360  ...           0   

   GCON05OWRI  GCON05RNOR  GCON06DCLY  GCON06OWRI  GCON06RBUC 

In [8]:
print(sc_election_gdf.columns.values)  

['UNIQUE_ID' 'COUNTYFP' 'County' 'Precinct' 'Prec_Code' 'CONG_DIST'
 'GCON01AODD' 'GCON01DAND' 'GCON01OWRI' 'GCON01RMAC' 'GCON02DLAR'
 'GCON02OWRI' 'GCON02RWIL' 'GCON03OWRI' 'GCON03RDUN' 'GCON04OWRI'
 'GCON04RTIM' 'GCON05DHUN' 'GCON05GGAI' 'GCON05OWRI' 'GCON05RNOR'
 'GCON06DCLY' 'GCON06OWRI' 'GCON06RBUC' 'GCON07DSCO' 'GCON07OWRI'
 'GCON07RFRY' 'geometry']


In [9]:
columns_to_ignore = ['UNIQUE_ID', 'CONG_DIST','COUNTYFP', 'County', 'Prec_Code', 'Precinct', 'geometry']

In [10]:
columns_to_use = [col for col in sc_election_gdf.columns if col not in columns_to_ignore]

In [11]:
print(columns_to_use)

['GCON01AODD', 'GCON01DAND', 'GCON01OWRI', 'GCON01RMAC', 'GCON02DLAR', 'GCON02OWRI', 'GCON02RWIL', 'GCON03OWRI', 'GCON03RDUN', 'GCON04OWRI', 'GCON04RTIM', 'GCON05DHUN', 'GCON05GGAI', 'GCON05OWRI', 'GCON05RNOR', 'GCON06DCLY', 'GCON06OWRI', 'GCON06RBUC', 'GCON07DSCO', 'GCON07OWRI', 'GCON07RFRY']


In [16]:
sc_election_processed_gdf = process_precinct_votes(sc_election_gdf, columns_to_use, columns_to_ignore)

In [17]:
print(sc_election_processed_gdf[['TOT_REP', 'TOT_DEM', 'TOT_VOT', 'LEAN']].head())

   TOT_REP  TOT_DEM  TOT_VOT        LEAN
0      540      286      826  Republican
1      682      479     1161  Republican
2       85       69      154  Republican
3      296      144      440  Republican
4      360      189      549  Republican


In [18]:
print(sc_election_processed_gdf[['LEAN']].value_counts())

LEAN      
Republican    1672
Democratic     597
Unknown          8
Name: count, dtype: int64


In [19]:
print(sc_election_processed_gdf.columns.values)

['UNIQUE_ID' 'CONG_DIST' 'COUNTYFP' 'County' 'Prec_Code' 'Precinct'
 'geometry' 'TOT_REP' 'TOT_DEM' 'TOT_VOT' 'LEAN' 'GCON01RMAC' 'GCON02RWIL'
 'GCON03RDUN' 'GCON04RTIM' 'GCON05RNOR' 'GCON06RBUC' 'GCON07RFRY'
 'GCON01DAND' 'GCON02DLAR' 'GCON05DHUN' 'GCON06DCLY' 'GCON07DSCO']


In [20]:
print(sc_election_processed_gdf.columns.values)

['UNIQUE_ID' 'CONG_DIST' 'COUNTYFP' 'County' 'Prec_Code' 'Precinct'
 'geometry' 'TOT_REP' 'TOT_DEM' 'TOT_VOT' 'LEAN' 'GCON01RMAC' 'GCON02RWIL'
 'GCON03RDUN' 'GCON04RTIM' 'GCON05RNOR' 'GCON06RBUC' 'GCON07RFRY'
 'GCON01DAND' 'GCON02DLAR' 'GCON05DHUN' 'GCON06DCLY' 'GCON07DSCO']


In [21]:
#drop geometry column
sc_election_processed_gdf = sc_election_processed_gdf.drop(columns='geometry')

In [22]:
with open('states/south_carolina/election/sc_election.json', 'w') as json_file:
    json.dump(sc_election_processed_gdf.to_dict(orient='records'), json_file, indent=4)

#### Preprocess state-wide gov election

In [23]:
sc_gov_election_df = gpd.read_file('raw/precincts/sc_2022_gen_prec/sc_2022_gen_st_prec/sc_2022_gen_st_prec.shp')

In [24]:
sc_gov_election_df = generate_unique_id(sc_gov_election_df, county_col='County', prec_code_col='Prec_Code')

In [25]:
print(sc_gov_election_df.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'Precinct' 'Prec_Code' 'G22A1NO'
 'G22A1YES' 'G22A2NO' 'G22A2YES' 'G22AGRCNEL' 'G22AGRGEDM' 'G22AGROWRI'
 'G22AGRRWEA' 'G22ATGOWRI' 'G22ATGRWIL' 'G22COMOWRI' 'G22COMRECK'
 'G22GOVDCUN' 'G22GOVLREE' 'G22GOVOWRI' 'G22GOVRMCM' 'G22SOSDBUT'
 'G22SOSOWRI' 'G22SOSRHAM' 'G22SUPAELL' 'G22SUPDELL' 'G22SUPGMIC'
 'G22SUPOWRI' 'G22SUPRWEA' 'G22TREAWOR' 'G22TREOWRI' 'G22TRERLOF'
 'G22USSDMAT' 'G22USSOWRI' 'G22USSRSCO' 'geometry']


In [26]:
columns_to_keep = ['UNIQUE_ID', 'COUNTYFP', 'County', 'Precinct', 'Prec_Code']
columns_to_keep += [col for col in sc_gov_election_df.columns if col.startswith('G22GOV')]
sc_gov_election_df = sc_gov_election_df[columns_to_keep]

In [27]:
print(sc_gov_election_df.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'Precinct' 'Prec_Code' 'G22GOVDCUN'
 'G22GOVLREE' 'G22GOVOWRI' 'G22GOVRMCM']


In [28]:
columns_to_ignore = ['UNIQUE_ID','COUNTYFP', 'County', 'Prec_Code', 'Precinct']

In [29]:
columns_to_use = [col for col in sc_gov_election_df.columns if col not in columns_to_ignore]

In [30]:
print(columns_to_use)

['G22GOVDCUN', 'G22GOVLREE', 'G22GOVOWRI', 'G22GOVRMCM']


In [31]:
sc_gov_election_processed_df = process_precinct_votes(sc_gov_election_df, columns_to_use, columns_to_ignore)

In [32]:
print(sc_gov_election_processed_df[['TOT_REP', 'TOT_DEM', 'TOT_VOT', 'LEAN']].head())

   TOT_REP  TOT_DEM  TOT_VOT        LEAN
0      718      257      975  Republican
1      224      404      628  Democratic
2      336      238      574  Republican
3      316      121      437  Republican
4      599      123      722  Republican


In [33]:
print(sc_gov_election_processed_df[['LEAN']].value_counts())

LEAN      
Republican    1526
Democratic     727
Unknown          8
Name: count, dtype: int64


In [34]:
print(sc_gov_election_processed_df.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'Prec_Code' 'Precinct' 'TOT_REP'
 'TOT_DEM' 'TOT_VOT' 'LEAN' 'G22GOVRMCM' 'G22GOVDCUN']


In [35]:
# drop 'COUNTYFP', 'County'
sc_gov_election_processed_df = sc_gov_election_processed_df.drop(columns=[ 'G22GOVRMCM', 'G22GOVDCUN'])

In [36]:
print(sc_gov_election_processed_df.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'Prec_Code' 'Precinct' 'TOT_REP'
 'TOT_DEM' 'TOT_VOT' 'LEAN']


In [37]:
# remove 'Prec_Code' 'Precinct'
sc_gov_election_processed_df = sc_gov_election_processed_df.drop(columns=['Prec_Code', 'Precinct'])

In [38]:
with open('states/south_carolina/election/sc_election_gov_22.json', 'w') as json_file:
    json.dump(sc_gov_election_processed_df.to_dict(orient='records'), json_file, indent=4)

## MD election preprocessing

### Election preprocessing for General election Cong District

In [867]:
def create_unique_id(row):
    # Remove 'County', capitalize, and replace spaces with underscores
    county = row["County Name"].replace("County", "").strip().upper().replace(" ", "_")
    # Remove the leading zero from the Election District
    district_precinct = row["Election District - Precinct"]
    district_precinct = district_precinct[1:]
    # Concatenate to form the UNIQUE_ID
    return f"{county}_PRECINCT_{district_precinct}"

In [ ]:
# needed format "UNIQUE_ID": "ABBEVILLE_PRECINCT_001",
        # "Prec_Code": "001",
        # "Precinct": "ABBEVILLE NO. 1",
        # "TOT_REP": 718,
        # "TOT_DEM": 257,
        # "TOT_VOT": 975,
        # "LEAN": "Republican"

In [678]:
md_precincts_gdf = gpd.read_file('states/maryland/geodata/maryland_precincts.geojson')


In [583]:
print(md_precincts_gdf.head())

                     NAME  NUMBER                 UNIQUE_ID  \
0  HOWARD PRECINCT 06-001  06-001  0_HOWARD_PRECINCT_06-001   
1  HOWARD PRECINCT 05-023  05-023  1_HOWARD_PRECINCT_05-023   
2  HOWARD PRECINCT 05-018  05-018  2_HOWARD_PRECINCT_05-018   
3  HOWARD PRECINCT 05-017  05-017  3_HOWARD_PRECINCT_05-017   
4  HOWARD PRECINCT 05-020  05-020  4_HOWARD_PRECINCT_05-020   

                                            geometry  
0  POLYGON Z ((-76.83025 39.14757 0.00000, -76.83...  
1  POLYGON Z ((-76.89107 39.22616 0.00000, -76.89...  
2  POLYGON Z ((-76.88639 39.20934 0.00000, -76.88...  
3  POLYGON Z ((-76.88670 39.20810 0.00000, -76.88...  
4  POLYGON Z ((-76.91361 39.14963 0.00000, -76.91...  


In [774]:
# load csv
md_election_df = pd.read_csv('raw/precincts/HOUSE_precinct_general.csv')

In [775]:
print(md_election_df.head())

                    precinct    office party_detailed party_simplified  \
0  061110097164_003026019037  US HOUSE       DEMOCRAT         DEMOCRAT   
1  061110097164_003026019037  US HOUSE     REPUBLICAN       REPUBLICAN   
2  061110097164_003026019037  US HOUSE       DEMOCRAT         DEMOCRAT   
3  061110097164_003026019037  US HOUSE     REPUBLICAN       REPUBLICAN   
4  061110097168_003026019037  US HOUSE       DEMOCRAT         DEMOCRAT   

           mode  votes county_name  county_fips jurisdiction_name  \
0      ABSENTEE     32     VENTURA       6111.0           VENTURA   
1      ABSENTEE      1     VENTURA       6111.0           VENTURA   
2  NOT ABSENTEE      1     VENTURA       6111.0           VENTURA   
3  NOT ABSENTEE      3     VENTURA       6111.0           VENTURA   
4      ABSENTEE    303     VENTURA       6111.0           VENTURA   

   jurisdiction_fips  ...       state  special writein  state_po state_fips  \
0             6111.0  ...  CALIFORNIA    False   False       

In [776]:
#filter state_po == MD
md_election_df = md_election_df[md_election_df['state_po'] == 'MD']

In [777]:
print(md_election_df.columns.values)

['precinct' 'office' 'party_detailed' 'party_simplified' 'mode' 'votes'
 'county_name' 'county_fips' 'jurisdiction_name' 'jurisdiction_fips'
 'candidate' 'district' 'dataverse' 'year' 'stage' 'state' 'special'
 'writein' 'state_po' 'state_fips' 'state_cen' 'state_ic' 'date'
 'readme_check' 'magnitude']


In [778]:
print(md_election_df['county_name'].value_counts())

county_name
BALTIMORE CITY     6365
MONTGOMERY         5510
PRINCE GEORGE'S    4545
BALTIMORE          4140
ANNE ARUNDEL       2925
HOWARD             2440
FREDERICK          1880
HARFORD            1335
WASHINGTON         1325
ALLEGANY            925
WICOMICO            810
CARROLL             655
CHARLES             645
DORCHESTER          600
ST MARY'S           540
GARRETT             475
CECIL               420
CALVERT             345
SOMERSET            345
WORCESTER           300
TALBOT              180
QUEEN ANNE'S        165
KENT                150
CAROLINE            120
Name: count, dtype: int64


In [808]:
md_election_df['NAME'] = md_election_df['county_name'].str.replace(' ', '_') + '_PRECINCT_' + md_election_df['precinct'].astype(str)

columns_order = ['NAME'] + [col for col in md_election_df.columns if col != 'NAME']
md_election_df = md_election_df[columns_order]

In [809]:
print(md_election_df.head())    

                               NAME precinct    office party_detailed  \
52046      WICOMICO_PRECINCT_01-001   01-001  US HOUSE       DEMOCRAT   
52047    DORCHESTER_PRECINCT_01-001   01-001  US HOUSE       DEMOCRAT   
52048      SOMERSET_PRECINCT_01-001   01-001  US HOUSE       DEMOCRAT   
52049      CAROLINE_PRECINCT_01-001   01-001  US HOUSE       DEMOCRAT   
52050  QUEEN_ANNE'S_PRECINCT_01-001   01-001  US HOUSE       DEMOCRAT   

      party_simplified          mode  votes   county_name  county_fips  \
52046         DEMOCRAT  2ND ABSENTEE      6      WICOMICO      24045.0   
52047         DEMOCRAT  2ND ABSENTEE     36    DORCHESTER      24019.0   
52048         DEMOCRAT  2ND ABSENTEE     37      SOMERSET      24039.0   
52049         DEMOCRAT  2ND ABSENTEE     43      CAROLINE      24011.0   
52050         DEMOCRAT  2ND ABSENTEE     55  QUEEN ANNE'S      24035.0   

      jurisdiction_name  ...  special writein  state_po state_fips  state_cen  \
52046          WICOMICO  ...    Fal

In [810]:
print(md_election_df[md_election_df['NAME'] == 'HOWARD_PRECINCT_05-023'])

                         NAME precinct    office party_detailed  \
79781  HOWARD_PRECINCT_05-023   05-023  US HOUSE       DEMOCRAT   
79782  HOWARD_PRECINCT_05-023   05-023  US HOUSE       DEMOCRAT   
79783  HOWARD_PRECINCT_05-023   05-023  US HOUSE       DEMOCRAT   
79784  HOWARD_PRECINCT_05-023   05-023  US HOUSE       DEMOCRAT   
79785  HOWARD_PRECINCT_05-023   05-023  US HOUSE       DEMOCRAT   
79786  HOWARD_PRECINCT_05-023   05-023  US HOUSE     REPUBLICAN   
79787  HOWARD_PRECINCT_05-023   05-023  US HOUSE     REPUBLICAN   
79788  HOWARD_PRECINCT_05-023   05-023  US HOUSE     REPUBLICAN   
79789  HOWARD_PRECINCT_05-023   05-023  US HOUSE     REPUBLICAN   
79790  HOWARD_PRECINCT_05-023   05-023  US HOUSE     REPUBLICAN   

      party_simplified          mode  votes county_name  county_fips  \
79781         DEMOCRAT  2ND ABSENTEE    198      HOWARD      24027.0   
79782         DEMOCRAT      ABSENTEE    467      HOWARD      24027.0   
79783         DEMOCRAT  EARLY VOTING    288   

In [811]:
print(md_election_df.columns.values)

['NAME' 'precinct' 'office' 'party_detailed' 'party_simplified' 'mode'
 'votes' 'county_name' 'county_fips' 'jurisdiction_name'
 'jurisdiction_fips' 'candidate' 'district' 'dataverse' 'year' 'stage'
 'state' 'special' 'writein' 'state_po' 'state_fips' 'state_cen'
 'state_ic' 'date' 'readme_check' 'magnitude' 'variable']


In [812]:
print(md_election_df[['NAME','office', 'party_detailed', 'candidate', 'writein', 'district', 'votes']].head())


                               NAME    office party_detailed  candidate  \
52046      WICOMICO_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52047    DORCHESTER_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52048      SOMERSET_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52049      CAROLINE_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52050  QUEEN_ANNE'S_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   

       writein  district  votes  
52046    False         1      6  
52047    False         1     36  
52048    False         1     37  
52049    False         1     43  
52050    False         1     55  


In [813]:
md_election_df = md_election_df[md_election_df['writein'] != True]

In [814]:
print(md_election_df[['NAME','office', 'party_detailed', 'candidate', 'district', 'votes']].head())

                               NAME    office party_detailed  candidate  \
52046      WICOMICO_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52047    DORCHESTER_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52048      SOMERSET_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52049      CAROLINE_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   
52050  QUEEN_ANNE'S_PRECINCT_01-001  US HOUSE       DEMOCRAT  MIA MASON   

       district  votes  
52046         1      6  
52047         1     36  
52048         1     37  
52049         1     43  
52050         1     55  


In [815]:
md_election_df['variable'] = (
    'GCON' + 
    md_election_df['district'].astype(str).str.zfill(2) + 
    md_election_df['party_detailed'].str[0] + 
    md_election_df['candidate'].apply(lambda x: ''.join([name[0] for name in x.split()]))
)

In [816]:
md_election_df_filtered = md_election_df[['NAME', 'precinct', 'county_name', 'county_fips', 'variable', 'votes', 'district']]
print(md_election_df_filtered)

                               NAME precinct   county_name  county_fips  \
52046      WICOMICO_PRECINCT_01-001   01-001      WICOMICO      24045.0   
52047    DORCHESTER_PRECINCT_01-001   01-001    DORCHESTER      24019.0   
52048      SOMERSET_PRECINCT_01-001   01-001      SOMERSET      24039.0   
52049      CAROLINE_PRECINCT_01-001   01-001      CAROLINE      24011.0   
52050  QUEEN_ANNE'S_PRECINCT_01-001   01-001  QUEEN ANNE'S      24035.0   
...                             ...      ...           ...          ...   
89131     FREDERICK_PRECINCT_26-002   26-002     FREDERICK      24021.0   
89132     FREDERICK_PRECINCT_26-002   26-002     FREDERICK      24021.0   
89133     FREDERICK_PRECINCT_26-002   26-002     FREDERICK      24021.0   
89134     FREDERICK_PRECINCT_26-002   26-002     FREDERICK      24021.0   
89135     FREDERICK_PRECINCT_26-002   26-002     FREDERICK      24021.0   

         variable  votes  district  
52046   GCON01DMM      6         1  
52047   GCON01DMM     36 

In [817]:
print(len(md_election_df_filtered['NAME'].unique()))

2035


In [818]:
md_election_pivoted_df = md_election_df_filtered.pivot_table(
    index=['NAME'], 
    columns='variable',
    values='votes',
    aggfunc='sum'
).fillna(0).reset_index() 


In [819]:
district_mapping = md_election_df_filtered[['NAME', 'district']].drop_duplicates()
md_election_pivoted_df = md_election_pivoted_df.merge(district_mapping, on='NAME', how='left')

In [820]:
print(district_mapping)

                               NAME  district
52046      WICOMICO_PRECINCT_01-001         1
52047    DORCHESTER_PRECINCT_01-001         1
52048      SOMERSET_PRECINCT_01-001         1
52049      CAROLINE_PRECINCT_01-001         1
52050  QUEEN_ANNE'S_PRECINCT_01-001         1
...                             ...       ...
89041     FREDERICK_PRECINCT_24-002         8
89061     FREDERICK_PRECINCT_24-003         8
89081     FREDERICK_PRECINCT_24-006         8
89101     FREDERICK_PRECINCT_26-001         8
89121     FREDERICK_PRECINCT_26-002         8

[2035 rows x 2 columns]


In [821]:
print(md_election_pivoted_df.head())


                       NAME  GCON01DMM  GCON01RAH  GCON02DCDR  GCON02RJRS  \
0  ALLEGANY_PRECINCT_01-000        0.0        0.0         0.0         0.0   
1  ALLEGANY_PRECINCT_02-000        0.0        0.0         0.0         0.0   
2  ALLEGANY_PRECINCT_03-000        0.0        0.0         0.0         0.0   
3  ALLEGANY_PRECINCT_04-002        0.0        0.0         0.0         0.0   
4  ALLEGANY_PRECINCT_04-003        0.0        0.0         0.0         0.0   

   GCON03DJS  GCON03RCA  GCON04DAGB  GCON04RGEM  GCON05DSHH  GCON05RCP  \
0        0.0        0.0         0.0         0.0         0.0        0.0   
1        0.0        0.0         0.0         0.0         0.0        0.0   
2        0.0        0.0         0.0         0.0         0.0        0.0   
3        0.0        0.0         0.0         0.0         0.0        0.0   
4        0.0        0.0         0.0         0.0         0.0        0.0   

   GCON06DDJT  GCON06GGG  GCON06RNCP  GCON07DKM  GCON07RKK  GCON08DJR  \
0        72.0      

In [822]:
print(md_election_pivoted_df)

                           NAME  GCON01DMM  GCON01RAH  GCON02DCDR  GCON02RJRS  \
0      ALLEGANY_PRECINCT_01-000        0.0        0.0         0.0         0.0   
1      ALLEGANY_PRECINCT_02-000        0.0        0.0         0.0         0.0   
2      ALLEGANY_PRECINCT_03-000        0.0        0.0         0.0         0.0   
3      ALLEGANY_PRECINCT_04-002        0.0        0.0         0.0         0.0   
4      ALLEGANY_PRECINCT_04-003        0.0        0.0         0.0         0.0   
...                         ...        ...        ...         ...         ...   
2030  WORCESTER_PRECINCT_05-002      955.0     1592.0         0.0         0.0   
2031  WORCESTER_PRECINCT_06-001      921.0     1742.0         0.0         0.0   
2032  WORCESTER_PRECINCT_06-002      250.0      753.0         0.0         0.0   
2033  WORCESTER_PRECINCT_06-003      430.0     1115.0         0.0         0.0   
2034  WORCESTER_PRECINCT_07-001     1532.0     3005.0         0.0         0.0   

      GCON03DJS  GCON03RCA 

In [826]:
print(md_election_pivoted_df.columns.values)

['NAME' 'GCON01DMM' 'GCON01RAH' 'GCON02DCDR' 'GCON02RJRS' 'GCON03DJS'
 'GCON03RCA' 'GCON04DAGB' 'GCON04RGEM' 'GCON05DSHH' 'GCON05RCP'
 'GCON06DDJT' 'GCON06GGG' 'GCON06RNCP' 'GCON07DKM' 'GCON07RKK' 'GCON08DJR'
 'GCON08RGTC' 'district']


## Process voting population:

In [827]:
columns_to_ignore = ['NAME', 'district']
columns_to_use = [col for col in md_election_pivoted_df.columns if col not in columns_to_ignore]

In [828]:
print(columns_to_use)

['GCON01DMM', 'GCON01RAH', 'GCON02DCDR', 'GCON02RJRS', 'GCON03DJS', 'GCON03RCA', 'GCON04DAGB', 'GCON04RGEM', 'GCON05DSHH', 'GCON05RCP', 'GCON06DDJT', 'GCON06GGG', 'GCON06RNCP', 'GCON07DKM', 'GCON07RKK', 'GCON08DJR', 'GCON08RGTC']


In [829]:
md_election_processed_df = process_precinct_votes(md_election_pivoted_df, columns_to_use, columns_to_ignore)


In [830]:
print(md_election_processed_df.columns.values)

['NAME' 'district' 'TOT_REP' 'TOT_DEM' 'TOT_VOT' 'LEAN' 'GCON01RAH'
 'GCON02RJRS' 'GCON03RCA' 'GCON04RGEM' 'GCON05RCP' 'GCON06RNCP'
 'GCON07RKK' 'GCON08RGTC' 'GCON01DMM' 'GCON02DCDR' 'GCON03DJS'
 'GCON04DAGB' 'GCON05DSHH' 'GCON06DDJT' 'GCON07DKM' 'GCON08DJR']


In [831]:
print(md_election_processed_df.head())

                       NAME  district  TOT_REP  TOT_DEM  TOT_VOT        LEAN  \
0  ALLEGANY_PRECINCT_01-000         6    406.0     72.0    478.0  Republican   
1  ALLEGANY_PRECINCT_02-000         6    398.0     93.0    491.0  Republican   
2  ALLEGANY_PRECINCT_03-000         6    419.0     89.0    508.0  Republican   
3  ALLEGANY_PRECINCT_04-002         6    188.0    144.0    332.0  Republican   
4  ALLEGANY_PRECINCT_04-003         6    354.0    248.0    602.0  Republican   

   GCON01RAH  GCON02RJRS  GCON03RCA  GCON04RGEM  ...  GCON07RKK  GCON08RGTC  \
0        0.0         0.0        0.0         0.0  ...        0.0         0.0   
1        0.0         0.0        0.0         0.0  ...        0.0         0.0   
2        0.0         0.0        0.0         0.0  ...        0.0         0.0   
3        0.0         0.0        0.0         0.0  ...        0.0         0.0   
4        0.0         0.0        0.0         0.0  ...        0.0         0.0   

   GCON01DMM  GCON02DCDR  GCON03DJS  GCON04D

In [832]:
print(md_election_processed_df[['LEAN']].value_counts())

LEAN      
Democratic    1392
Republican     596
Unknown         47
Name: count, dtype: int64


In [833]:
# get UNIQUE_ID where district == 4

print(md_election_processed_df[md_election_processed_df['district'] == 4])

                                 NAME  district  TOT_REP  TOT_DEM  TOT_VOT  \
64       ANNE_ARUNDEL_PRECINCT_02-002         4    517.0    626.0   1143.0   
75       ANNE_ARUNDEL_PRECINCT_02-013         4    709.0    693.0   1402.0   
80       ANNE_ARUNDEL_PRECINCT_02-018         4    354.0    767.0   1121.0   
82       ANNE_ARUNDEL_PRECINCT_02-020         4    727.0    883.0   1610.0   
83       ANNE_ARUNDEL_PRECINCT_02-021         4   1280.0   1170.0   2450.0   
...                               ...       ...      ...      ...      ...   
1795  PRINCE_GEORGE'S_PRECINCT_20-097         4      0.0      0.0      0.0   
1796  PRINCE_GEORGE'S_PRECINCT_20-098         4      0.0      0.0      0.0   
1797  PRINCE_GEORGE'S_PRECINCT_20-099         4      0.0      0.0      0.0   
1802  PRINCE_GEORGE'S_PRECINCT_21-005         4    172.0   1000.0   1172.0   
1811  PRINCE_GEORGE'S_PRECINCT_21-014         4    151.0    716.0    867.0   

            LEAN  GCON01RAH  GCON02RJRS  GCON03RCA  GCON04RGEM 

In [834]:
#rename NAME to UNIQUE_ID
md_election_processed_df = md_election_processed_df.rename(columns={'NAME': 'UNIQUE_ID'})

In [835]:
with open('states/maryland/election/md_election_cd.json', 'w') as json_file:
    json.dump(md_election_processed_df.to_dict(orient='records'), json_file, indent=4)

### Election preprocessing for Govenor

In [885]:
md_election_csv = pd.read_csv('raw/census_block/election/GG22_AllPrecincts.csv')

In [886]:
print(md_election_csv.columns.values)

['County' 'County Name' 'Election District - Precinct' 'Congressional'
 'Legislative' 'Office Name' 'Office District' 'Candidate Name' 'Party'
 'Winner' 'Write-In?' 'Early Votes' 'Early Votes Against'
 'Election Night Votes' 'Election Night Votes Against'
 'Mail-In Ballot 1 Votes' 'Mail-In Ballot 1 Votes Against'
 'Provisional Votes' 'Provisional Votes Against' 'Mail-In Ballot 2 Votes'
 'Mail-In Ballot 2 Votes Against ']


In [887]:
# filter Office Name to get only "Governor / Lt. Governor"
md_gov_election_df = md_election_csv[md_election_csv['Office Name'] == 'Governor / Lt. Governor']

In [888]:
md_gov_election_df["UNIQUE_ID"] = md_gov_election_df.apply(create_unique_id, axis=1)

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_96987/2367602659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  md_gov_election_df["UNIQUE_ID"] = md_gov_election_df.apply(create_unique_id, axis=1)


In [889]:
# filter 'Party' to get DEM and REP
md_gov_election_df = md_gov_election_df[md_gov_election_df['Party'].isin(['DEM', 'REP'])]

In [893]:
print(md_gov_election_df)

        County       County Name Election District - Precinct  Congressional  \
0            1   Allegany County                      001-000              6   
1            1   Allegany County                      001-000              6   
56           1   Allegany County                      002-000              6   
57           1   Allegany County                      002-000              6   
112          1   Allegany County                      003-000              6   
...        ...               ...                          ...            ...   
122913      24  Worcester County                      006-002              1   
122958      24  Worcester County                      006-003              1   
122959      24  Worcester County                      006-003              1   
123004      24  Worcester County                      007-001              1   
123005      24  Worcester County                      007-001              1   

       Legislative              Office 

In [899]:
print(md_gov_election_df['Party'].value_counts())

Party
REP    2074
DEM    2074
Name: count, dtype: int64


In [896]:
# G22GOVDCUN
md_gov_election_df['variable'] = (
    'G22GOV' + 
    md_gov_election_df['Party'].str[0] + 
    md_gov_election_df['Candidate Name'].apply(lambda x: ''.join([name[0] for name in ' '.join(x.split()[:2]).split()]))
)


In [3]:
print(md_gov_election_df['Candidate Name'].value_counts())

NameError: name 'md_gov_election_df' is not defined

In [897]:
print(md_gov_election_df['UNIQUE_ID'].head())

0      ALLEGANY_PRECINCT_01-000
1      ALLEGANY_PRECINCT_01-000
56     ALLEGANY_PRECINCT_02-000
57     ALLEGANY_PRECINCT_02-000
112    ALLEGANY_PRECINCT_03-000
Name: UNIQUE_ID, dtype: object


In [900]:
print(md_gov_election_df['variable'].value_counts())

variable
G22GOVRDC    2074
G22GOVDWM    2074
Name: count, dtype: int64


In [902]:
md_gov_election_df["votes"] = (
    md_gov_election_df["Early Votes"] +
    md_gov_election_df["Election Night Votes"] +
    md_gov_election_df["Mail-In Ballot 1 Votes"] +
    md_gov_election_df["Provisional Votes"] +
    md_gov_election_df["Mail-In Ballot 2 Votes"]
)

In [903]:
print(md_gov_election_df[['UNIQUE_ID', 'variable', 'votes']].head())

                    UNIQUE_ID   variable  votes
0    ALLEGANY_PRECINCT_01-000  G22GOVRDC    274
1    ALLEGANY_PRECINCT_01-000  G22GOVDWM     57
56   ALLEGANY_PRECINCT_02-000  G22GOVRDC    311
57   ALLEGANY_PRECINCT_02-000  G22GOVDWM     53
112  ALLEGANY_PRECINCT_03-000  G22GOVRDC    324


In [904]:
print(md_gov_election_df.columns.values)

['County' 'County Name' 'Election District - Precinct' 'Congressional'
 'Legislative' 'Office Name' 'Office District' 'Candidate Name' 'Party'
 'Winner' 'Write-In?' 'Early Votes' 'Early Votes Against'
 'Election Night Votes' 'Election Night Votes Against'
 'Mail-In Ballot 1 Votes' 'Mail-In Ballot 1 Votes Against'
 'Provisional Votes' 'Provisional Votes Against' 'Mail-In Ballot 2 Votes'
 'Mail-In Ballot 2 Votes Against ' 'UNIQUE_ID' 'variable' 'votes']


In [907]:
# filter 'UNIQUE_ID' 'Election District - Precinct' 'variable' 'votes'
md_gov_election_df_filtered = md_gov_election_df[['UNIQUE_ID', 'Election District - Precinct', 'variable', 'votes']]
#rename 'Election District - Precinct' to 'Prec_Code'
md_gov_election_df_filtered = md_gov_election_df_filtered.rename(columns={'Election District - Precinct': 'Prec_Code'})

In [908]:
print(md_gov_election_df_filtered.head())

                    UNIQUE_ID Prec_Code   variable  votes
0    ALLEGANY_PRECINCT_01-000   001-000  G22GOVRDC    274
1    ALLEGANY_PRECINCT_01-000   001-000  G22GOVDWM     57
56   ALLEGANY_PRECINCT_02-000   002-000  G22GOVRDC    311
57   ALLEGANY_PRECINCT_02-000   002-000  G22GOVDWM     53
112  ALLEGANY_PRECINCT_03-000   003-000  G22GOVRDC    324


In [909]:
md_gov_election_pivoted_df = md_gov_election_df_filtered.pivot_table(
    index=['UNIQUE_ID'], 
    columns='variable',
    values='votes',
    aggfunc='sum'
).fillna(0).reset_index() 

In [910]:
print(md_gov_election_pivoted_df.head())

variable                 UNIQUE_ID  G22GOVDWM  G22GOVRDC
0         ALLEGANY_PRECINCT_01-000         57        274
1         ALLEGANY_PRECINCT_02-000         53        311
2         ALLEGANY_PRECINCT_03-000         54        324
3         ALLEGANY_PRECINCT_04-002         56        129
4         ALLEGANY_PRECINCT_04-003        154        228


In [913]:
columns_to_ignore = ['UNIQUE_ID']
columns_to_use = [col for col in md_gov_election_pivoted_df.columns if col not in columns_to_ignore]

In [914]:
print(columns_to_use)

['G22GOVDWM', 'G22GOVRDC']


In [915]:
md_gov_election_processed_df = process_precinct_votes(md_gov_election_pivoted_df, columns_to_use, columns_to_ignore)

In [916]:
print(md_gov_election_processed_df.columns.values)

['UNIQUE_ID' 'TOT_REP' 'TOT_DEM' 'TOT_VOT' 'LEAN' 'G22GOVRDC' 'G22GOVDWM']


In [917]:
print(md_gov_election_processed_df['LEAN'].value_counts())

LEAN
Democratic    1521
Republican     541
Unknown         12
Name: count, dtype: int64


In [918]:
print(md_gov_election_processed_df.head())

variable                 UNIQUE_ID  TOT_REP  TOT_DEM  TOT_VOT        LEAN  \
0         ALLEGANY_PRECINCT_01-000      274       57      331  Republican   
1         ALLEGANY_PRECINCT_02-000      311       53      364  Republican   
2         ALLEGANY_PRECINCT_03-000      324       54      378  Republican   
3         ALLEGANY_PRECINCT_04-002      129       56      185  Republican   
4         ALLEGANY_PRECINCT_04-003      228      154      382  Republican   

variable  G22GOVRDC  G22GOVDWM  
0               274         57  
1               311         53  
2               324         54  
3               129         56  
4               228        154  


In [919]:
#drop G22GOVRDC G22GOVDWM
md_gov_election_processed_df = md_gov_election_processed_df.drop(columns=['G22GOVRDC', 'G22GOVDWM'])

In [920]:
with open('states/maryland/election/md_election_gov_22.json', 'w') as json_file:
    json.dump(md_gov_election_processed_df.to_dict(orient='records'), json_file, indent=4)